# DSPy Tool Use: Uma Abordagem Didática Completa

Este notebook oferece uma exploração abrangente dos conceitos de **Tool Use** no DSPy, demonstrando como construir sistemas de IA que podem interagir com ferramentas externas de forma inteligente e autônoma.

## 📚 Referências
- **DSPy Official Documentation**: https://dspy.ai/
- **DSPy GitHub Repository**: https://github.com/stanfordnlp/dspy
- **Tool Use Tutorial**: https://dspy.ai/tutorials/tool_use/
- **Groq API**: https://groq.com/

## 🎯 Objetivos de Aprendizagem
1. Compreender os fundamentos do Tool Use em sistemas de IA
2. Implementar ferramentas customizadas para DSPy
3. Construir agentes ReAct (Reasoning + Acting)
4. Explorar padrões avançados de composição de ferramentas
5. Otimizar performance de sistemas multi-ferramentas

## 🔧 Setup e Configuração

Começamos configurando o ambiente com Groq como provedor de LLM.

In [ ]:
import dspy
import os
from dotenv import load_dotenv
load_dotenv()
import json
import requests
import math
import random
import time
from typing import List, Dict, Any, Optional, Union
from datetime import datetime, timedelta
from dataclasses import dataclass
import pandas as pd
import numpy as np

# Para visualização
import matplotlib.pyplot as plt
import seaborn as sns

# Configuração de estilo
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

## Observabilidade com Langfuse


In [ ]:
from langfuse import get_client
 
langfuse = get_client()
 
# Verify connection
if langfuse.auth_check():
    print("Langfuse client is authenticated and ready!")
else:
    print("Authentication failed. Please check your credentials and host.")


## Enable Tracing for DSPy


In [ ]:
from openinference.instrumentation.dspy import DSPyInstrumentor
DSPyInstrumentor().instrument()


In [ ]:
# Configuração do Groq LLM
# Defina sua GROQ_API_KEY no ambiente:
# export GROQ_API_KEY="your-groq-api-key-here"

# Configurar DSPy com Groq
groq_llm = dspy.LM(
    model='groq/llama-3.3-70b-versatile',  # ou 'groq/mixtral-8x7b-32768'
    api_key=os.getenv('GROQ_API_KEY'),
    max_tokens=1000
)

dspy.configure(lm=groq_llm)

print("✅ DSPy configurado com Groq LLM")
print(f"📡 Modelo: {groq_llm.model}")

## 🧠 Teoria: O que é Tool Use?

### Conceitos Fundamentais

**Tool Use** (Uso de Ferramentas) é um paradigma onde sistemas de IA podem:

1. **🔍 Identificar** quando uma tarefa requer ferramentas externas
2. **🎯 Selecionar** a ferramenta apropriada para a situação
3. **⚡ Executar** a ferramenta com parâmetros corretos
4. **🧩 Interpretar** os resultados para continuar o raciocínio
5. **🔄 Iterar** até completar a tarefa

### Arquitetura ReAct

O padrão **ReAct** (Reasoning + Acting) alterna entre:
- **Reasoning**: Planejamento e análise
- **Acting**: Execução de ações/ferramentas

```
Thought → Action → Observation → Thought → Action → ...
```

## 🛠️ Parte 1: Criando Ferramentas Básicas

Vamos começar criando ferramentas simples e entender como elas funcionam.

In [ ]:
# Ferramenta 1: Calculadora Matemática
def calculator(expression: str) -> str:
    """
    Avalia expressões matemáticas de forma segura.
    
    Args:
        expression: Expressão matemática como string (ex: "2 + 3 * 4")
    
    Returns:
        Resultado da operação como string
    """
    try:
        # Lista de operações permitidas (segurança)
        allowed_chars = set('0123456789+-*/().= ')
        if not all(c in allowed_chars for c in expression):
            return "Erro: Caracteres não permitidos na expressão"
        
        # Avalia a expressão
        result = eval(expression)
        return f"Resultado: {result}"
    except Exception as e:
        return f"Erro no cálculo: {str(e)}"

# Teste da ferramenta
print("🧮 Testando Calculator:")
print(calculator("15 * 7 + 3"))
print(calculator("sqrt(16)"))  # Deve dar erro - função não permitida

In [ ]:
# Ferramenta 2: Gerador de Dados Simulados
def generate_data(data_type: str, count: int = 10) -> str:
    """
    Gera dados simulados para diferentes propósitos.
    
    Args:
        data_type: Tipo de dados ('sales', 'users', 'temperatures')
        count: Número de registros a gerar
    
    Returns:
        Dados em formato JSON
    """
    try:
        if data_type == 'sales':
            data = []
            for i in range(count):
                data.append({
                    'id': i + 1,
                    'product': f'Product_{random.choice(["A", "B", "C", "D"])}',
                    'amount': round(random.uniform(10, 1000), 2),
                    'date': (datetime.now() - timedelta(days=random.randint(0, 30))).strftime('%Y-%m-%d')
                })
        
        elif data_type == 'users':
            names = ['João', 'Maria', 'Pedro', 'Ana', 'Carlos', 'Lucia']
            data = []
            for i in range(count):
                data.append({
                    'id': i + 1,
                    'name': random.choice(names),
                    'age': random.randint(18, 65),
                    'email': f'user{i+1}@example.com'
                })
        
        elif data_type == 'temperatures':
            data = []
            for i in range(count):
                data.append({
                    'day': i + 1,
                    'temperature': round(random.uniform(15, 35), 1),
                    'humidity': round(random.uniform(30, 90), 1)
                })
        
        else:
            return f"Erro: Tipo de dados '{data_type}' não suportado"
        
        return json.dumps(data, indent=2, ensure_ascii=False)
    
    except Exception as e:
        return f"Erro na geração de dados: {str(e)}"

# Teste da ferramenta
print("📊 Testando Data Generator:")
sales_data = generate_data('sales', 3)
print(sales_data)

In [ ]:
# Ferramenta 3: Analisador de Texto
def text_analyzer(text: str, analysis_type: str = 'basic') -> str:
    """
    Analisa características de um texto.
    
    Args:
        text: Texto a ser analisado
        analysis_type: Tipo de análise ('basic', 'detailed', 'sentiment')
    
    Returns:
        Análise do texto em formato JSON
    """
    try:
        words = text.split()
        sentences = text.split('.')
        
        result = {
            'character_count': len(text),
            'word_count': len(words),
            'sentence_count': len([s for s in sentences if s.strip()]),
        }
        
        if analysis_type == 'detailed':
            # Análise mais detalhada
            word_lengths = [len(word.strip('.,!?;:')) for word in words]
            result.update({
                'avg_word_length': round(sum(word_lengths) / len(word_lengths) if word_lengths else 0, 2),
                'longest_word': max(words, key=len) if words else '',
                'unique_words': len(set(word.lower().strip('.,!?;:') for word in words))
            })
        
        elif analysis_type == 'sentiment':
            # Análise de sentimento simples baseada em palavras-chave
            positive_words = ['bom', 'ótimo', 'excelente', 'feliz', 'alegre', 'positivo']
            negative_words = ['ruim', 'péssimo', 'triste', 'negativo', 'problema', 'erro']
            
            text_lower = text.lower()
            pos_count = sum(1 for word in positive_words if word in text_lower)
            neg_count = sum(1 for word in negative_words if word in text_lower)
            
            if pos_count > neg_count:
                sentiment = 'Positivo'
            elif neg_count > pos_count:
                sentiment = 'Negativo'
            else:
                sentiment = 'Neutro'
            
            result.update({
                'sentiment': sentiment,
                'positive_indicators': pos_count,
                'negative_indicators': neg_count
            })
        
        return json.dumps(result, indent=2, ensure_ascii=False)
    
    except Exception as e:
        return f"Erro na análise: {str(e)}"

# Teste da ferramenta
print("📝 Testando Text Analyzer:")
sample_text = "Este é um texto muito bom para testar nossa ferramenta. Ela funciona de forma excelente!"
print(text_analyzer(sample_text, 'sentiment'))

## 🤖 Parte 2: Criando um Agente ReAct Simples

Agora vamos criar um agente que pode usar essas ferramentas.

In [ ]:
# Definindo a assinatura do nosso agente
class ToolUseAgent(dspy.Signature):
    """
    Você é um assistente inteligente que pode usar ferramentas para responder perguntas.
    
    Ferramentas disponíveis:
    1. calculator(expression) - Calcula expressões matemáticas
    2. generate_data(data_type, count) - Gera dados simulados
    3. text_analyzer(text, analysis_type) - Analisa características de texto
    
    Sempre explique seu raciocínio e como você chegou à resposta.
    """
    
    question = dspy.InputField(desc="Pergunta ou tarefa do usuário")
    answer = dspy.OutputField(desc="Resposta completa usando as ferramentas quando necessário")

# Criando o agente ReAct
tools = [calculator, generate_data, text_analyzer]

agent = dspy.ReAct(
    signature=ToolUseAgent,
    tools=tools,
    max_iters=5
)

print("🤖 Agente ReAct criado com sucesso!")
print(f"🔧 Ferramentas disponíveis: {len(tools)}")

## 🧪 Parte 3: Testando o Agente com Diferentes Cenários

Vamos testar nosso agente com diferentes tipos de perguntas.

In [ ]:
# Teste 1: Pergunta matemática
print("🧮 TESTE 1: Matemática")
print("="*50)

question1 = "Qual é o resultado de 25 * 13 + 47?"
print(f"Pergunta: {question1}")
print("\nResposta do Agente:")

try:
    result1 = agent(question=question1)
    print(result1.answer)
except Exception as e:
    print(f"Erro: {e}")

print("\n" + "="*50)

In [ ]:
# Teste 2: Geração e análise de dados
print("📊 TESTE 2: Análise de Dados")
print("="*50)

question2 = "Gere 5 registros de vendas e me diga qual é o produto mais comum"
print(f"Pergunta: {question2}")
print("\nResposta do Agente:")

try:
    result2 = agent(question=question2)
    print(result2.answer)
except Exception as e:
    print(f"Erro: {e}")

print("\n" + "="*50)

In [ ]:
# Teste 3: Análise de texto
print("📝 TESTE 3: Análise de Texto")
print("="*50)

question3 = "Analise o sentimento desta frase: 'O produto é muito bom, mas o atendimento foi péssimo'"
print(f"Pergunta: {question3}")
print("\nResposta do Agente:")

try:
    result3 = agent(question=question3)
    print(result3.answer)
except Exception as e:
    print(f"Erro: {e}")

print("\n" + "="*50)

## 🔬 Parte 4: Ferramentas Avançadas - API Integration

Vamos criar ferramentas mais sofisticadas que interagem com APIs externas.

In [ ]:
# Ferramenta 4: Consulta de CEP (usando API pública)
def cep_lookup(cep: str) -> str:
    """
    Consulta informações de um CEP brasileiro.
    
    Args:
        cep: CEP no formato 12345-678 ou 12345678
    
    Returns:
        Informações do endereço em JSON
    """
    try:
        # Remove formatação do CEP
        clean_cep = cep.replace('-', '').replace('.', '')
        
        if len(clean_cep) != 8 or not clean_cep.isdigit():
            return "Erro: CEP deve ter 8 dígitos"
        
        # Consulta a API pública do ViaCEP
        url = f"https://viacep.com.br/ws/{clean_cep}/json/"
        response = requests.get(url, timeout=5)
        
        if response.status_code == 200:
            data = response.json()
            if 'erro' in data:
                return "Erro: CEP não encontrado"
            return json.dumps(data, indent=2, ensure_ascii=False)
        else:
            return f"Erro na consulta: Status {response.status_code}"
    
    except requests.RequestException:
        return "Erro: Não foi possível conectar à API de CEP"
    except Exception as e:
        return f"Erro: {str(e)}"

# Teste da ferramenta
print("🏠 Testando CEP Lookup:")
print(cep_lookup("01310-100"))  # CEP da Av. Paulista, SP

In [ ]:
# Ferramenta 5: Conversor de Moedas (usando API pública)
def currency_converter(amount: float, from_currency: str, to_currency: str) -> str:
    """
    Converte valores entre moedas.
    
    Args:
        amount: Valor a ser convertido
        from_currency: Moeda de origem (ex: 'USD')
        to_currency: Moeda de destino (ex: 'BRL')
    
    Returns:
        Resultado da conversão em JSON
    """
    try:
        # Para este exemplo, vamos usar taxas fixas simuladas
        # Em produção, você usaria uma API real como exchangerate-api.com
        
        rates = {
            'USD': {'BRL': 5.20, 'EUR': 0.85, 'GBP': 0.75},
            'BRL': {'USD': 0.19, 'EUR': 0.16, 'GBP': 0.14},
            'EUR': {'USD': 1.18, 'BRL': 6.12, 'GBP': 0.88}
        }
        
        from_currency = from_currency.upper()
        to_currency = to_currency.upper()
        
        if from_currency == to_currency:
            converted_amount = amount
        elif from_currency in rates and to_currency in rates[from_currency]:
            rate = rates[from_currency][to_currency]
            converted_amount = amount * rate
        else:
            return f"Erro: Conversão {from_currency} → {to_currency} não suportada"
        
        result = {
            'original_amount': amount,
            'from_currency': from_currency,
            'to_currency': to_currency,
            'converted_amount': round(converted_amount, 2),
            'rate': rates.get(from_currency, {}).get(to_currency, 1.0),
            'timestamp': datetime.now().isoformat()
        }
        
        return json.dumps(result, indent=2, ensure_ascii=False)
    
    except Exception as e:
        return f"Erro na conversão: {str(e)}"

# Teste da ferramenta
print("💱 Testando Currency Converter:")
print(currency_converter(100, 'USD', 'BRL'))

## 🚀 Parte 5: Agente Avançado com Múltiplas Ferramentas

Vamos criar um agente mais poderoso com todas as ferramentas.

In [ ]:
# Assinatura do agente avançado
class AdvancedToolAgent(dspy.Signature):
    """
    Você é um assistente avançado com acesso a múltiplas ferramentas especializadas.
    
    Ferramentas disponíveis:
    1. calculator(expression) - Cálculos matemáticos
    2. generate_data(data_type, count) - Geração de dados simulados
    3. text_analyzer(text, analysis_type) - Análise de texto
    4. cep_lookup(cep) - Consulta de CEP brasileiro
    5. currency_converter(amount, from_currency, to_currency) - Conversão de moedas
    
    Combine ferramentas quando necessário para fornecer respostas completas e úteis.
    Sempre explique seu processo de raciocínio.
    """
    
    request = dspy.InputField(desc="Solicitação ou pergunta complexa do usuário")
    response = dspy.OutputField(desc="Resposta detalhada usando as ferramentas apropriadas")

# Criando o agente avançado
advanced_tools = [calculator, generate_data, text_analyzer, cep_lookup, currency_converter]

advanced_agent = dspy.ReAct(
    signature=AdvancedToolAgent,
    tools=advanced_tools,
    max_iters=8
)

print("🚀 Agente Avançado criado!")
print(f"🔧 Total de ferramentas: {len(advanced_tools)}")

## 🎯 Parte 6: Cenários de Teste Avançados

Vamos testar o agente com tarefas que requerem múltiplas ferramentas.

In [ ]:
# Teste Avançado 1: Análise de Negócio
print("💼 TESTE AVANÇADO 1: Análise de Negócio")
print("="*60)

request1 = """
Preciso fazer uma análise de vendas. 
Gere 8 registros de vendas e calcule:
1. O valor total das vendas
2. A média de vendas por produto
3. Converta o total para EUR
"""

print(f"Solicitação: {request1.strip()}")
print("\n" + "-"*40)
print("Resposta do Agente:")

try:
    result1 = advanced_agent(request=request1)
    print(result1.response)
except Exception as e:
    print(f"Erro: {e}")

print("\n" + "="*60)

In [ ]:
# Teste Avançado 2: Análise Geográfica e Financeira
print("🌍 TESTE AVANÇADO 2: Análise Geo-Financeira")
print("="*60)

request2 = """
Consulte o CEP 04038-001 e depois calcule quanto custaria 
enviar um produto que custa R$ 150 para essa região, 
considerando uma taxa de entrega de 8% do valor do produto.
Também converta o valor total para USD.
"""

print(f"Solicitação: {request2.strip()}")
print("\n" + "-"*40)
print("Resposta do Agente:")

try:
    result2 = advanced_agent(request=request2)
    print(result2.response)
except Exception as e:
    print(f"Erro: {e}")

print("\n" + "="*60)

In [ ]:
# Teste Avançado 3: Análise Multi-Modal
print("📊 TESTE AVANÇADO 3: Análise Multi-Modal")
print("="*60)

request3 = """
Analise este feedback de cliente: 
'O produto chegou muito rápido e a qualidade é excelente, mas o preço está um pouco alto'.

Depois gere 3 dados de temperatura e calcule a média.
Se a média for maior que 25°C, calcule quanto seria 100 BRL em EUR.
"""

print(f"Solicitação: {request3.strip()}")
print("\n" + "-"*40)
print("Resposta do Agente:")

try:
    result3 = advanced_agent(request=request3)
    print(result3.response)
except Exception as e:
    print(f"Erro: {e}")

print("\n" + "="*60)

## 📈 Parte 7: Análise de Performance e Otimização

Vamos analisar como nosso sistema se comporta e identificar oportunidades de melhoria.

In [ ]:
# Função para medir performance
def measure_performance(agent, requests, iterations=3):
    """
    Mede a performance do agente em múltiplos testes.
    """
    results = []
    
    for i, request in enumerate(requests):
        print(f"🔄 Testando requisição {i+1}/{len(requests)}")
        
        times = []
        successes = 0
        
        for iteration in range(iterations):
            try:
                start_time = time.time()
                result = agent(request=request)
                end_time = time.time()
                
                times.append(end_time - start_time)
                successes += 1
                
            except Exception as e:
                print(f"  ❌ Erro na iteração {iteration+1}: {e}")
                times.append(None)
        
        # Calcula métricas
        valid_times = [t for t in times if t is not None]
        avg_time = sum(valid_times) / len(valid_times) if valid_times else 0
        success_rate = successes / iterations
        
        results.append({
            'request_id': i + 1,
            'request': request[:50] + '...' if len(request) > 50 else request,
            'avg_time': round(avg_time, 2),
            'success_rate': round(success_rate, 2),
            'total_iterations': iterations
        })
    
    return results

# Testes de performance
test_requests = [
    "Calcule 15 * 23 + 7",
    "Gere 3 dados de vendas",
    "Analise o sentimento: 'Estou muito feliz hoje!'",
    "Converta 50 USD para BRL"
]

print("⚡ Iniciando análise de performance...")
performance_results = measure_performance(advanced_agent, test_requests, iterations=2)

# Exibe resultados
df_performance = pd.DataFrame(performance_results)
print("\n📊 Resultados de Performance:")
print(df_performance.to_string(index=False))

In [ ]:
# Visualização dos resultados
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Gráfico de tempo de resposta
ax1.bar(df_performance['request_id'], df_performance['avg_time'], 
        color='skyblue', alpha=0.7)
ax1.set_xlabel('ID da Requisição')
ax1.set_ylabel('Tempo Médio (segundos)')
ax1.set_title('Tempo de Resposta por Requisição')
ax1.grid(True, alpha=0.3)

# Gráfico de taxa de sucesso
ax2.bar(df_performance['request_id'], df_performance['success_rate'], 
        color='lightgreen', alpha=0.7)
ax2.set_xlabel('ID da Requisição')
ax2.set_ylabel('Taxa de Sucesso')
ax2.set_title('Taxa de Sucesso por Requisição')
ax2.set_ylim(0, 1.1)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Estatísticas gerais
print("\n📈 Estatísticas Gerais:")
print(f"⏱️  Tempo médio geral: {df_performance['avg_time'].mean():.2f}s")
print(f"✅ Taxa de sucesso geral: {df_performance['success_rate'].mean():.2%}")
print(f"⚡ Requisição mais rápida: {df_performance['avg_time'].min():.2f}s")
print(f"🐌 Requisição mais lenta: {df_performance['avg_time'].max():.2f}s")

## 🔧 Parte 8: Padrões Avançados e Melhores Práticas

Vamos explorar técnicas avançadas para otimizar nossos agentes.

In [ ]:
# Padrão 1: Ferramentas com Cache
class CachedTool:
    """
    Wrapper que adiciona cache às ferramentas para melhorar performance.
    """
    def __init__(self, tool_func, cache_size=100):
        self.tool_func = tool_func
        self.cache = {}
        self.cache_size = cache_size
        self.hit_count = 0
        self.miss_count = 0
    
    def __call__(self, *args, **kwargs):
        # Cria chave do cache
        cache_key = str(args) + str(sorted(kwargs.items()))
        
        if cache_key in self.cache:
            self.hit_count += 1
            return self.cache[cache_key]
        
        # Executa a ferramenta
        result = self.tool_func(*args, **kwargs)
        
        # Armazena no cache
        if len(self.cache) >= self.cache_size:
            # Remove item mais antigo
            oldest_key = next(iter(self.cache))
            del self.cache[oldest_key]
        
        self.cache[cache_key] = result
        self.miss_count += 1
        
        return result
    
    def get_stats(self):
        total = self.hit_count + self.miss_count
        hit_rate = self.hit_count / total if total > 0 else 0
        return {
            'hits': self.hit_count,
            'misses': self.miss_count,
            'hit_rate': hit_rate,
            'cache_size': len(self.cache)
        }

# Aplicando cache às ferramentas
cached_calculator = CachedTool(calculator)
cached_converter = CachedTool(currency_converter)

print("🗄️ Cache implementado nas ferramentas")

# Teste do cache
print("\nTestando cache:")
print("1ª execução:", cached_calculator("10 + 5"))
print("2ª execução (cache):", cached_calculator("10 + 5"))
print("Stats:", cached_calculator.get_stats())

In [ ]:
# Padrão 2: Validação de Entrada
def validated_tool(validation_func):
    """
    Decorator para adicionar validação às ferramentas.
    """
    def decorator(tool_func):
        def wrapper(*args, **kwargs):
            # Valida entradas
            validation_result = validation_func(*args, **kwargs)
            if validation_result is not True:
                return f"Erro de validação: {validation_result}"
            
            # Executa a ferramenta
            return tool_func(*args, **kwargs)
        
        wrapper.__name__ = tool_func.__name__
        wrapper.__doc__ = tool_func.__doc__
        return wrapper
    return decorator

# Validadores
def validate_calculator_input(expression):
    if not isinstance(expression, str):
        return "Expressão deve ser uma string"
    if len(expression) > 100:
        return "Expressão muito longa"
    return True

def validate_currency_input(amount, from_currency, to_currency):
    if not isinstance(amount, (int, float)) or amount < 0:
        return "Valor deve ser um número positivo"
    if not isinstance(from_currency, str) or not isinstance(to_currency, str):
        return "Moedas devem ser strings"
    return True

# Aplicando validação
@validated_tool(validate_calculator_input)
def safe_calculator(expression: str) -> str:
    return calculator(expression)

@validated_tool(validate_currency_input)
def safe_currency_converter(amount: float, from_currency: str, to_currency: str) -> str:
    return currency_converter(amount, from_currency, to_currency)

print("🛡️ Validação implementada")

# Teste da validação
print("\nTestando validação:")
print("Entrada válida:", safe_calculator("2 + 2"))
print("Entrada inválida:", safe_currency_converter(-10, "USD", "BRL"))

In [ ]:
# Padrão 3: Logging e Monitoramento
import logging
from functools import wraps

# Configurar logging
logging.basicConfig(level=logging.INFO, 
                   format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger('DSPy_Tools')

def logged_tool(tool_func):
    """
    Decorator para adicionar logging às ferramentas.
    """
    @wraps(tool_func)
    def wrapper(*args, **kwargs):
        start_time = time.time()
        
        logger.info(f"Executando {tool_func.__name__} com args={args}, kwargs={kwargs}")
        
        try:
            result = tool_func(*args, **kwargs)
            execution_time = time.time() - start_time
            
            logger.info(f"{tool_func.__name__} concluída em {execution_time:.3f}s")
            return result
            
        except Exception as e:
            execution_time = time.time() - start_time
            logger.error(f"{tool_func.__name__} falhou após {execution_time:.3f}s: {e}")
            raise
    
    return wrapper

# Aplicando logging
@logged_tool
def monitored_calculator(expression: str) -> str:
    return calculator(expression)

print("📊 Logging implementado")

# Teste do logging
print("\nTestando logging:")
result = monitored_calculator("15 * 3")
print(f"Resultado: {result}")

## 📚 Parte 9: Casos de Uso Avançados

Vamos implementar cenários do mundo real mais complexos.

In [ ]:
# Caso de Uso 1: Sistema de Análise de E-commerce
def ecommerce_analyzer(task: str) -> str:
    """
    Analisa dados de e-commerce de forma integrada.
    """
    try:
        if "vendas" in task.lower():
            # Gera dados de vendas
            sales_data = generate_data('sales', 10)
            
            # Parseia para calcular total
            import json
            sales_list = json.loads(sales_data)
            total = sum(item['amount'] for item in sales_list)
            
            # Calcula estatísticas
            avg = total / len(sales_list)
            
            # Converte para outras moedas
            usd_total = currency_converter(total, 'BRL', 'USD')
            
            return f"""
Análise de E-commerce:
- Total de vendas: R$ {total:.2f}
- Média por venda: R$ {avg:.2f}
- Em USD: {usd_total}
- Número de transações: {len(sales_list)}
"""
        
        elif "feedback" in task.lower():
            # Gera feedback fictício
            feedbacks = [
                "Produto excelente, entrega rápida!",
                "Qualidade boa mas preço alto",
                "Atendimento péssimo, produto ok",
                "Muito bom, recomendo!"
            ]
            
            results = []
            for feedback in feedbacks:
                analysis = text_analyzer(feedback, 'sentiment')
                results.append(analysis)
            
            return f"Análise de {len(feedbacks)} feedbacks completada"
        
        else:
            return "Tarefa não reconhecida"
            
    except Exception as e:
        return f"Erro na análise: {e}"

# Teste do sistema
print("🛒 Sistema de E-commerce:")
print(ecommerce_analyzer("Analise as vendas do mês"))

In [ ]:
# Caso de Uso 2: Dashboard de Métricas
def create_dashboard_data():
    """
    Cria dados para um dashboard executivo.
    """
    # Gera diferentes tipos de dados
    sales_data = json.loads(generate_data('sales', 15))
    user_data = json.loads(generate_data('users', 8))
    temp_data = json.loads(generate_data('temperatures', 7))
    
    # Calcula KPIs
    total_sales = sum(item['amount'] for item in sales_data)
    avg_age = sum(user['age'] for user in user_data) / len(user_data)
    avg_temp = sum(day['temperature'] for day in temp_data) / len(temp_data)
    
    # Converte vendas para diferentes moedas
    sales_usd = json.loads(currency_converter(total_sales, 'BRL', 'USD'))
    sales_eur = json.loads(currency_converter(total_sales, 'BRL', 'EUR'))
    
    dashboard = {
        'kpis': {
            'total_sales_brl': round(total_sales, 2),
            'total_sales_usd': sales_usd['converted_amount'],
            'total_sales_eur': sales_eur['converted_amount'],
            'avg_customer_age': round(avg_age, 1),
            'avg_temperature': round(avg_temp, 1),
            'total_transactions': len(sales_data),
            'total_customers': len(user_data)
        },
        'trends': {
            'revenue_per_customer': round(total_sales / len(user_data), 2),
            'revenue_per_transaction': round(total_sales / len(sales_data), 2)
        }
    }
    
    return dashboard

# Cria dashboard
dashboard_data = create_dashboard_data()

print("📊 Dashboard Executivo:")
print("\n🎯 KPIs Principais:")
for key, value in dashboard_data['kpis'].items():
    print(f"  • {key.replace('_', ' ').title()}: {value}")

print("\n📈 Tendências:")
for key, value in dashboard_data['trends'].items():
    print(f"  • {key.replace('_', ' ').title()}: R$ {value}")

## 🎓 Conclusões e Próximos Passos

### O que Aprendemos

1. **🔧 Fundamentos**: Como criar e integrar ferramentas customizadas no DSPy
2. **🤖 Agentes ReAct**: Implementação de agentes que alternam entre reasoning e acting
3. **🚀 Padrões Avançados**: Cache, validação, logging e monitoramento
4. **📊 Casos Reais**: Aplicações em e-commerce, dashboards e análise de dados
5. **⚡ Performance**: Técnicas de otimização e medição de performance

### Arquitetura do Sistema Tool Use

```
┌─────────────────┐    ┌─────────────────┐    ┌─────────────────┐
│   User Input    │ -> │   DSPy Agent    │ -> │  Tool Selection │
└─────────────────┘    └─────────────────┘    └─────────────────┘
                              │                         │
                              v                         v
┌─────────────────┐    ┌─────────────────┐    ┌─────────────────┐
│ Final Response  │ <- │   LLM Reasoning │ <- │ Tool Execution  │
└─────────────────┘    └─────────────────┘    └─────────────────┘
```

### Melhores Práticas Identificadas

✅ **Validação de Entrada**: Sempre validar parâmetros das ferramentas
✅ **Tratamento de Erros**: Implementar fallbacks e mensagens de erro claras
✅ **Cache Inteligente**: Usar cache para operações repetitivas
✅ **Logging Completo**: Monitorar execução e performance
✅ **Documentação Clara**: Ferramentas bem documentadas para o LLM

### Próximos Passos Sugeridos

1. **🔗 Integração com APIs Reais**: Conectar com serviços como Weather API, Stock API
2. **🧠 Ferramentas de ML**: Implementar ferramentas que usam modelos treinados
3. **📁 Persistência**: Adicionar banco de dados para histórico de operações
4. **🌐 Interface Web**: Criar interface web para interação com os agentes
5. **🔄 Workflows Complexos**: Implementar pipelines multi-agentes

### Recursos para Aprofundamento

- **Documentação Oficial**: https://dspy.ai/
- **Repositório GitHub**: https://github.com/stanfordnlp/dspy
- **Exemplos da Comunidade**: https://github.com/topics/dspy
- **Groq Documentation**: https://console.groq.com/docs

---

**🎉 Parabéns!** Você completou uma exploração abrangente do Tool Use no DSPy. Este conhecimento pode ser aplicado para criar sistemas de IA mais poderosos e versáteis.